In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

# ✅ CONFIGURATION — Update these paths as needed

# Folder containing masks (stored on Google Drive)
mask_dir = "/content/drive/MyDrive/Khushi_drive/All_Masks_PNG"

# Folder containing image chips (stored locally in Colab, e.g., from previous step)
image_dir = "/content"  # <-- Make sure this exists and contains .png images

# Output location for YOLO dataset (back to Drive or local)
save_dir = "/content/drive/MyDrive/yolo_airstrip_dataset"  # or "/content/yolo_dataset"

# Bounding box padding in pixels
PADDING_PIXELS = 3

# Output folders
img_out_dir = os.path.join(save_dir, "images/train")
lbl_out_dir = os.path.join(save_dir, "labels/train")
os.makedirs(img_out_dir, exist_ok=True)
os.makedirs(lbl_out_dir, exist_ok=True)


In [ ]:
!pip install --quiet rasterio opencv-python-headless tqdm numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.9 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

def get_all_image_files(folder, extensions=["*.png", "*.PNG"]):
    files = []
    for ext in extensions:
        files.extend(glob(os.path.join(folder, ext)))
    return sorted(files)

def read_mask_with_padding(mask_path, padding=PADDING_PIXELS):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None

    coords = np.column_stack(np.where(mask > 0))
    if coords.size == 0:
        return None

    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    h, w = mask.shape

    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(w - 1, x_max + padding)
    y_max = min(h - 1, y_max + padding)

    return x_min, y_min, x_max, y_max

def convert_to_yolo_format(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    bbox_width = (x_max - x_min) / img_width
    bbox_height = (y_max - y_min) / img_height
    return x_center, y_center, bbox_width, bbox_height

def process_png_image(image_path):
    img_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img_bgr is None:
        return None
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)


In [ ]:
mask_paths = get_all_image_files(mask_dir)
converted, skipped = 0, 0

print(f"🔍 Found {len(mask_paths)} mask files")
print(f"📏 Padding used: {PADDING_PIXELS} pixels")

for mask_path in tqdm(mask_paths, desc="Processing masks"):
    mask_filename = os.path.basename(mask_path)

    # Remove class prefix (e.g., 0_id_123.png → id_123)
    if mask_filename[0].isdigit() and mask_filename[1] == "_":
        base_name = os.path.splitext(mask_filename.split("_", 1)[1])[0]
    else:
        base_name = os.path.splitext(mask_filename)[0]

    # Match corresponding image
    image_path = os.path.join(image_dir, base_name + ".png")
    if not os.path.exists(image_path):
        print(f"⚠️ Missing image: {base_name}.png")
        skipped += 1
        continue

    # Load the image
    img_rgb = process_png_image(image_path)
    if img_rgb is None:
        print(f"⚠️ Failed to load image: {image_path}")
        skipped += 1
        continue

    # Save image (regardless of mask)
    out_img_path = os.path.join(img_out_dir, base_name + ".png")
    cv2.imwrite(out_img_path, img_rgb[..., ::-1])  # RGB → BGR

    # Get bounding box from mask (if any)
    bbox_coords = read_mask_with_padding(mask_path)

    # Prepare label path
    out_label_path = os.path.join(lbl_out_dir, base_name + ".txt")

    if bbox_coords:
        h, w = img_rgb.shape[:2]
        x_center, y_center, bbox_width, bbox_height = convert_to_yolo_format(bbox_coords, w, h)
        yolo_label = f"0 {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        with open(out_label_path, "w") as f:
            f.write(yolo_label)
    else:
        # Write empty .txt file for negative sample
        with open(out_label_path, "w") as f:
            f.write("")

    converted += 1

print(f"\n✅ Converted: {converted}")
print(f"⚠️ Skipped (missing images only): {skipped}")
print(f"📁 Saved images: {img_out_dir}")
print(f"📁 Saved labels: {lbl_out_dir}")


🔍 Found 890 mask files
📏 Padding used: 3 pixels


Processing masks: 100%|██████████| 890/890 [02:12<00:00,  6.70it/s]


✅ Converted: 890
⚠️ Skipped (missing images only): 0
📁 Saved images: /content/drive/MyDrive/yolo_airstrip_dataset/images/train
📁 Saved labels: /content/drive/MyDrive/yolo_airstrip_dataset/labels/train


In [ ]:
import os

# Set the directory where your mask files are
mask_dir = "/content/drive/MyDrive/Khushi_drive/All_Masks_PNG"  # change if needed

# Get all filenames in the mask directory
all_mask_files = os.listdir(mask_dir)

# Initialize counters
from collections import Counter
prefix_counts = Counter()

for fname in all_mask_files:
    if fname[0].isdigit() and fname[1] == "_":
        prefix_counts[fname[0]] += 1

# Print summary
for label in sorted(prefix_counts):
    print(f"🔢 Files starting with {label}_ : {prefix_counts[label]}")

print(f"\n📦 Total counted: {sum(prefix_counts.values())}")


🔢 Files starting with 0_ : 288
🔢 Files starting with 1_ : 589

📦 Total counted: 877


In [ ]:
!ls /content/


drive	     id_1248.png  id_1491.png  id_306.png  id_538.png  id_761.png
id_1000.png  id_1252.png  id_1492.png  id_309.png  id_539.png  id_762.png
id_1001.png  id_1254.png  id_1493.png  id_30.png   id_53.png   id_763.png
id_1002.png  id_1255.png  id_1494.png  id_310.png  id_540.png  id_764.png
id_1003.png  id_1256.png  id_1496.png  id_311.png  id_541.png  id_765.png
id_1005.png  id_1258.png  id_1497.png  id_312.png  id_542.png  id_766.png
id_1006.png  id_1260.png  id_1498.png  id_313.png  id_543.png  id_769.png
id_1008.png  id_1261.png  id_149.png   id_314.png  id_544.png  id_76.png
id_1009.png  id_1262.png  id_1500.png  id_316.png  id_545.png  id_770.png
id_100.png   id_1263.png  id_1501.png  id_317.png  id_546.png  id_771.png
id_1011.png  id_1265.png  id_1502.png  id_31.png   id_547.png  id_772.png
id_1012.png  id_1266.png  id_1503.png  id_320.png  id_548.png  id_773.png
id_1013.png  id_1267.png  id_1504.png  id_323.png  id_549.png  id_778.png
id_1014.png  id_1268.png  id_1505.png  id

In [ ]:
yaml_content = f"""# YOLOv8 Dataset
path: {save_dir}
train: images/train
val: images/val  # Optional - you can split manually
test: images/test  # Optional

# Classes
nc: 1
names: ['airstrip']
"""

yaml_path = os.path.join(save_dir, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"✅ dataset.yaml saved to: {yaml_path}")


In [ ]:
os.makedirs(os.path.join(save_dir, "images/val"), exist_ok=True)
os.makedirs(os.path.join(save_dir, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(save_dir, "images/test"), exist_ok=True)
os.makedirs(os.path.join(save_dir, "labels/test"), exist_ok=True)
